In [285]:
import requests
import yfinance as yf
import pandas as pd
from bs4 import BeautifulSoup
import html5lib
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [308]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=(
        "Historical Share Price", "Historical Revenue"), vertical_spacing=.3)
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-06-01']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True),
                  y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True),
                  y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
                      height=900,
                      title=stock,
                      xaxis_rangeslider_visible=True)
    fig.show()

In [315]:
stock = yf.Ticker('TSLA')
tesla_data = stock.history(period='max')
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [288]:
html_data = requests.get(
    'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm').text
soups = BeautifulSoup(html_data, 'html5lib')

In [316]:
tesla_revenue = pd.DataFrame(columns=['Date', 'Revenue'])
tesla_revenue_col = soups.find_all('tbody')[1]
for i in tesla_revenue_col:
    left_row = tesla_revenue_col.find_all('td')
    even_data = [td.text for j, td in enumerate(left_row) if j % 2 != 0]
    odd_data = [ts.text for k, ts in enumerate(left_row) if k % 2 == 0]
    # i.append({'Date':odd_data,'Revenue':even_data})
tesla_revenue['Revenue'] = even_data
tesla_revenue['Date'] = odd_data
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] !=""]
tesla_revenue.tail()

,Date,Revenue
48,2010-09-30,$31
49,2010-06-30,$28
50,2010-03-31,$21
52,2009-09-30,$46
53,2009-06-30,$27


In [317]:
game_stock=yf.Ticker('GME')
gme_data=game_stock.history(period='max')
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615921,1.662210,1.603296,1.662210,6892800,0.0,0.0


In [318]:
gme_revenue = pd.DataFrame(columns=['Date', 'Revenue'])
dates = []
revenues = []
html_data_2 = requests.get(
    ' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html').text
parser = BeautifulSoup(html_data_2, 'html.parser')
gme_revenue_col = parser.find_all('tbody')[1]

for row in gme_revenue_col.find_all('tr'):
    data = row.find_all('td')
    revenue_datas = [td.text for j, td in enumerate(left_row) if j % 2 != 0]
    date_datas = [ts.text for k, ts in enumerate(left_row) if k % 2 == 0]
    if len(data) >= 2:
        dates.append(data[0].text.strip())
        revenues.append(
            int(data[1].text.strip().replace(',', '').replace('$', '')))
revenues
gme_revenue['Date'] = dates
gme_revenue['Revenue'] = revenues
gme_revenue.tail()

,Date,Revenue
57,2006-01-31,1667
58,2005-10-31,534
59,2005-07-31,416
60,2005-04-30,475
61,2005-01-31,709


In [319]:
make_graph(tesla_data,tesla_revenue,'Tesla')

C:\Users\shahh\AppData\Local\Temp\ipykernel_23860\1477483173.py:6: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\shahh\AppData\Local\Temp\ipykernel_23860\1477483173.py:8: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



In [320]:
make_graph(gme_data, gme_revenue, 'GameStop')

C:\Users\shahh\AppData\Local\Temp\ipykernel_23860\1477483173.py:6: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\shahh\AppData\Local\Temp\ipykernel_23860\1477483173.py:8: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

